<a href="https://colab.research.google.com/github/CastHash532/fastmail-scraper/blob/main/fastmail_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Extraction**

!apt-get install python3-bs4

# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sys
import json

Headers = {
'Host': 'fastmaildz.com',
'Referer': 'https://fastmaildz.com/newfastmail/public/passportstatus',
}

client = requests.session()

# Retrieve the CSRF token first
client.get('https://fastmaildz.com/newfastmail/public/passportstatus')  # sets cookie
if 'csrftoken' in client.cookies:
    # Django 1.6 and up
    csrftoken = client.cookies['csrftoken']
else:
    # older versions
    csrftoken = client.cookies['csrf']

data = []

for d in range(1,23):
  for i in range(1,300):
    a="%03d" % i
    b="%02d" % d
    print('day: '+b+'  - n°:'+a)
    url = 'https://fastmaildz.com/newfastmail/public/packagestatus?searchoption=ndossier&destination=ALGI&nb='+b+'0921%2F0'+a+'%2F01&csrfmiddlewaretoken='+csrftoken
    response =  client.get(url, headers=Headers)
    if response.status_code !=  400:
      data.append(response.json())

cleaned_data =[]

for x in data:
  cleaned_data.append(
      [
       x[1]['courrier']['num_dossier'],
       x[1]['courrier']['date_precommande'],
       x[1]['courrier']['date_commande'],
      ]
  )

  resultat = pd.DataFrame(cleaned_data, columns= ['id','depot','fin'])

In [332]:
#@title Recherche
oper = 'depot' #@param ["depot","fin"]
date = '2021-09-09' #@param {type:"date"}
recherche = resultat.loc[resultat[oper] == date]
print('# Dossiers traités:')
print(recherche[oper].value_counts())
print('# Dossiers non traités:')
print(recherche['fin'].isna().sum())

# Dossiers traités:
2021-09-09    209
Name: depot, dtype: int64
# Dossiers non traités:
52
